# Pickle Model Upload Example

In [6]:
from numerapi import NumerAPI
import pandas as pd
import json

napi = NumerAPI()

feature_metadata = json.load(open("v4.1/features.json")) 
feature_cols = feature_metadata["feature_sets"]["small"]

training_data = pd.read_parquet("v4.1/train.parquet", columns= ["era"] + feature_cols + ["target"]) 

In [7]:
import lightgbm as lgb

try: 
    model = lgb.Booster(model_file='small_lgb.txt');
except lgb.basic.LightGBMError:
    model = lgb.LGBMRegressor(
      n_estimators=2000,
      learning_rate=0.01,
      max_depth=5,
      num_leaves=2 ** 5,
      colsample_bytree=0.1
    )
    model.fit(
      training_data[feature_cols],
      training_data["target"]
    );
    model.booster_.save_model("small_lgb.txt");

In [8]:
# Define any function that takes in a live_features and returns a submission
def predict(live_features: pd.DataFrame) -> pd.DataFrame:
    live_predictions = model.predict(live_features[feature_cols])
    submission = pd.Series(live_predictions, index=live_features.index)
    return submission.to_frame("prediction")

In [9]:
# Use the cloudpickle library to serialize your function and its dependencies
import cloudpickle
p = cloudpickle.dumps(predict)
with open("predict.pkl", "wb") as f:
    f.write(p)

In [ ]:
# Download the model file if running in Google Colab
try:
    from google.colab import files
    files.download('predict.pkl')
except:
    pass